In [1]:
%load_ext autoreload
%autoreload 2 

In [2]:
import torch
import numpy as np
import torch.optim as optim
from torch.utils import data
from sqnet import SQNet
from data import dataGen
from tqdm import tqdm
from jacobian import JacobianReg
import time
import crocoddyl


In [3]:
def hessian_train():
    # Tensor data for training
    positions, costs, grad1, grad2 = dataGen(size = 1000)
    
    # Generate a Neural Net
    net = SQNet(input_features = positions.shape[1], 
                 output_features = costs.shape[1],
                 n_hiddenunits = 256)

    criterion = torch.nn.MSELoss(reduction='sum')    
    optimizer = optim.Adam(net.parameters(), lr=0.001, weight_decay=1e-3)
    
    # Training 1
    for epoch in tqdm(range(500)):

        for i, j in zip(positions, grad2):
            
            i = i.reshape(1, 3)
            net.train()
            output = net.hessian(i)               


            loss = criterion(output, j)
            loss.requires_grad=True

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

    del positions, costs, grad1, grad2
    return net

In [4]:
net = torch.load("jacobian.pth")

In [ ]:
def jacobian_train(net):
    # Tensor data for training
    positions, costs, grad1, _ = dataGen(size = 2000)

    # Torch dataloader
    dataset = torch.utils.data.TensorDataset(positions,costs, grad1)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size = 1000) 


    # Initialize loss and optimizer
    criterion = torch.nn.MSELoss(reduction='sum')
    optimizer = optim.Adam(net.parameters(), lr=0.001, weight_decay=1e-3)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1000, gamma=0.95)
    lam1 = 0.1


    for epoch in tqdm(range(5000)):
        epoch_mse0 = 0.0
        epoch_mse1 = 0.0
        for i, (input, target0, target1) in enumerate(dataloader):  

            net.eval()

            output0 = net(input)               
            output1 = net.jacobian(input)

            net.train()
            mse0 = criterion(output0, target0)
            mse1 = criterion(output1, target1)


            epoch_mse0 += mse0.item() * len(input)
            epoch_mse1 += mse1.item() * len(input)

            loss = mse0 + lam1 * mse1 

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        scheduler.step()

        epoch_mse0 /= 3000
        epoch_mse1 /= 3000

        epoch_loss = epoch_mse0 + lam1*epoch_mse1 
        print('epoch', epoch,
          'lr', '{:.7f}'.format(optimizer.param_groups[0]['lr']),
          'mse0', '{:.5f}'.format(epoch_mse0),
          'mse1', '{:.5f}'.format(epoch_mse1),
          'Total Loss', '{:.5f}'.format(epoch_loss))

    
    return net


In [11]:
def train(net):
    # Tensor data for training
    positions, costs, _, _ = dataGen(size = 10000)

    # Torch dataloader
    dataset = torch.utils.data.TensorDataset(positions,costs)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size = 10000) 


    # Initialize loss and optimizer
    criterion = torch.nn.MSELoss(reduction='sum')
    optimizer = optim.Adam(net.parameters(), lr=0.001, weight_decay=1e-3)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1000, gamma=0.95)
    


    for epoch in tqdm(range(10000)):
        epoch_mse = 0.0
        
        for i, (input, target) in enumerate(dataloader):  

            net.eval()

            output = net(input)               
            

            net.train()
            mse = criterion(output, target)
           

            epoch_mse += mse.item() * len(input)
            

            loss = mse 

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        scheduler.step()

        epoch_mse /= 10000
       

        epoch_loss = epoch_mse 
        print('epoch', epoch,
          'lr', '{:.7f}'.format(optimizer.param_groups[0]['lr']),
          'mse', '{:.5f}'.format(epoch_mse))
          
          

    
    return net


In [12]:
net = train(net)

  0%|          | 3/10000 [00:00<25:22,  6.57it/s]

epoch 0 lr 0.0010000 mse 3906.92505
epoch 1 lr 0.0010000 mse 337391.65625
epoch 2 lr 0.0010000 mse 26917.19336


  0%|          | 6/10000 [00:00<20:58,  7.94it/s]

epoch 3 lr 0.0010000 mse 97200.57031
epoch 4 lr 0.0010000 mse 195046.43750
epoch 5 lr 0.0010000 mse 109312.89062


  0%|          | 9/10000 [00:00<18:42,  8.90it/s]

...

100%|█████████▉| 9998/10000 [16:53<00:00,  8.92it/s]

epoch 9996 lr 0.0006302 mse 1198.79578
epoch 9997 lr 0.0006302 mse 1199.03577


100%|██████████| 10000/10000 [16:54<00:00,  9.53it/s]

epoch 9998 lr 0.0006302 mse 1199.32324
epoch 9999 lr 0.0005987 mse 1199.32593


KeyboardInterrupt: 

In [13]:
torch.save(net, "valuenet.pth")